<a href="https://colab.research.google.com/github/Ran-Mewo/voice-changer/blob/main/Voice_Changer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[RVC on Colab](https://colab.research.google.com/github/w-okada/voice-changer/blob/master/Realtime_Voice_Changer_on_Colab.ipynb) but upto **11x faster!** <br>And it supports saving things over to Google Drive (Optional)<br><br>
**limitations being it's most likely running an older version of [w-okada's Voice Changer](https://github.com/w-okada/voice-changer) but it can be updated if you pass in an updated image*


---

## **⬇ VERY IMPORTANT ⬇**

You can use the following settings for better results:

If you're using a index: `f0: RMVPE_ONNX | Chunk: 112 or higher | Extra: 8192`<br>
If you're not using a index: `f0: RMVPE_ONNX | Chunk: 96 or higher | Extra: 16384`<br>
**Don't forget to select a T4 GPU in the GPU field, <b>NEVER</b> use CPU!
> Seems that PTH models performance better than ONNX for now, you can still try ONNX models and see if it satisfies you


*You can always [click here](https://github.com/YunaOneeChan/Voice-Changer-Settings) to check if these settings are up-to-date*

---

### <font color=red>⬇ Always use Colab GPU! (**IMPORTANT!**) ⬇</font>
You need to use a Colab GPU so the Voice Changer can work faster and better\
Use the menu above and click on **Runtime** » **Change runtime** » **Hardware acceleration** to select a GPU (**T4 is the free one**)

---
**Credits**<br>
Realtime Voice Changer by [w-okada](https://github.com/w-okada)<br>
Notebook files updated by [rafacasari](https://github.com/Rafacasari)<br>
Recommended settings by [YunaOneeChan](https://github.com/YunaOneeChan)<br>
This specific notebook by [Ran](https://github.com/Ran-Mewo)

**If it's an issue with RVC or the voice changer header over to** [AI Hub Discord](https://discord.gg/aihub) » ***#help-realtime-vc***

**If it's an issue with this notebook specifically or you'd like to request an update to the voice changer then message me at discord** [yourcuteneko](<https://discord.com/users/721351466828300350>)

In [ ]:
# @title Initialize docker by using [Docker in Colab](https://github.com/drengskapur/docker-in-colab) and setup the image<br>
# @markdown We don't really use docker for it's intended purpose, it's more of used as a massive compression algorithm<br><br>
# @markdown The default image used is [ranmewo/voice-changer](https://github.com/Ran-Mewo/voice-changer-docker)<br>You can change it to your own that was created in a [similar way](https://github.com/Ran-Mewo/voice-changer-docker/blob/master/Dockerfile)

### This part of the code is by Drengskapur and is licensed under the Apache License 2.0 (http://www.apache.org/licenses/LICENSE-2.0)
### https://github.com/drengskapur/docker-in-colab
def udocker_init():
    import os
    if not os.path.exists("/home/user"):
        !pip install udocker > /dev/null
        !udocker --allow-root install > /dev/null
        !useradd -m user > /dev/null
    print(f'Docker-in-Colab 1.1.0\n')
    print(f'Usage:     udocker("--help")')
    print(f'Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples')

    def execute(command: str):
        user_prompt = "\033[1;32muser@pc\033[0m"
        print(f"{user_prompt}$ udocker {command}")
        !su - user -c "udocker $command"

    return execute

udocker = udocker_init()
### END OF LICENSED CODE
Image = 'ranmewo/voice-changer:latest' # @param {type:"string"}

!pip install pyngrok
!apt-get install -y libportaudio2
udocker(f"--allow-root pull {Image}")
udocker(f"--allow-root create --name=voice-changer {Image}")

# Set up the python paths so it's linked to the container
import sys
import os
site_packages = "/home/user/.udocker/containers/voice-changer/ROOT/opt/conda/lib/python3.10/site-packages"
bin = "/home/user/.udocker/containers/voice-changer/ROOT/opt/conda/bin"
sys.path.append(site_packages)
os.environ['PYTHONPATH'] = f"{site_packages}:{os.environ.get('PYTHONPATH', '')}"
try:
    os.remove(f"{bin}/python") # Remove the python file so we don't accidentally execute it
except FileNotFoundError:
    pass
os.environ['PATH'] = f"{os.environ['PATH']}:{bin}"

Docker-in-Colab 1.1.0

Usage:     udocker("--help")
Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 65.3 kB of archives.
After this operation, 223 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Fetched 65.3 kB in 0s (218 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 121753 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.1_amd64.deb ...
Unpacking libportaudio2:amd64 (19.6.0-1.1) ...
Setting up libportaudio2:amd64 (19.6.0-1.1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.4) ...
/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symb

In [ ]:
# @title Link Google Drive (Optional)<br><br><font color=red>⬇(**IMPORTANT**)⬇</font><br>This will create a `voice-changer` folder in your drive, if anything breaks then please try deleting that folder first!
drive_path = '/content/drive/MyDrive/voice-changer'
system_path = '/home/user/.udocker/containers/voice-changer/ROOT/app'

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
os.makedirs(drive_path, exist_ok=True)

if not os.listdir(drive_path):
    # Copy the system_path directory to the drive_path directory
    !cp -r {system_path} {drive_path}

!rm -rf {system_path}
os.symlink(drive_path, system_path, True)


Mounted at /content/drive


In [ ]:
# @title Start Server **using ngrok**
# @markdown This cell will start the server, the first time that you run it will download the models, so it can take a while (~1-2 minutes)

# @markdown ---
# @markdown You'll need a ngrok account, but <font color=green>**it's free**</font> and easy to create!
# @markdown ---
# @markdown **1** - Create a <font color=green>**free**</font> account at [ngrok](https://dashboard.ngrok.com/signup) or **login with Google/Github account**\
# @markdown **2** - If you didn't logged in with Google/Github, you will need to **verify your e-mail**!\
# @markdown **3** - Click [this link](https://dashboard.ngrok.com/get-started/your-authtoken) to get your auth token, and place it here:
Token = '' # @param {type:"string"}
# @markdown **4** - *(optional)* Change to a region near to you or keep at United States if increase latency\
# @markdown `Default Region: us - United States (Ohio)`
Region = "us - United States (Ohio)" # @param ["ap - Asia/Pacific (Singapore)", "au - Australia (Sydney)","eu - Europe (Frankfurt)", "in - India (Mumbai)","jp - Japan (Tokyo)","sa - South America (Sao Paulo)", "us - United States (Ohio)"]

# @markdown **5** - *(optional)* Other options:
ClearConsole = True  # @param {type:"boolean"}
# @markdown **6** - Don't touch unless you know what you're doing!
PORT = 8000 # @param {type:"integer"}

%cd /home/user/.udocker/containers/voice-changer/ROOT/app
%cd app
%cd server

from pyngrok import conf, ngrok
MyConfig = conf.PyngrokConfig()
MyConfig.auth_token = Token
MyConfig.region = Region[0:2]
conf.set_default(MyConfig);

import subprocess, threading, time, socket, urllib.request

from pyngrok import ngrok
ngrokConnection = ngrok.connect(PORT)
public_url = ngrokConnection.public_url

from IPython.display import clear_output
def wait_for_server():
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', PORT))
        if result == 0:
            break
        sock.close()
    if ClearConsole:
        clear_output()
    print("--------- SERVER READY! ---------")
    print("Your server is available at:")
    print(public_url)
    print("---------------------------------")

threading.Thread(target=wait_for_server, daemon=True).start()
!python MMVCServerSIO.py \
  -p {PORT} \
  --https False \
  --content_vec_500 pretrain/checkpoint_best_legacy_500.pt \
  --content_vec_500_onnx pretrain/content_vec_500.onnx \
  --content_vec_500_onnx_on true \
  --hubert_base pretrain/hubert_base.pt \
  --hubert_base_jp pretrain/rinna_hubert_base_jp.pt \
  --hubert_soft pretrain/hubert/hubert-soft-0d54a1f4.pt \
  --nsf_hifigan pretrain/nsf_hifigan/model \
  --crepe_onnx_full pretrain/crepe_onnx_full.onnx \
  --crepe_onnx_tiny pretrain/crepe_onnx_tiny.onnx \
  --rmvpe pretrain/rmvpe.pt \
  --model_dir model_dir \
  --samples samples.json

ngrok.disconnect(ngrokConnection.public_url)

--------- SERVER READY! ---------
Your server is available at:
https://b8db-35-223-100-36.ngrok-free.app
---------------------------------
[2024-04-02 13:29:20] connet sid : nTULRgDbd_bQWcdwAAAC
[2024-04-02 13:29:20] connet sid : Aj5O7kzBjosPa7HNAAAD
Generated Strengths: for prev:(4096,), for cur:(4096,)
[Voice Changer] warming up... generating sola buffer.
[Voice Changer] update configuration: tran 11
[Voice Changer][RVC]: update_settings tran:11
[Voice Changer] update configuration: tran 10
[Voice Changer][RVC]: update_settings tran:10
[Voice Changer] update configuration: tran 9
[Voice Changer][RVC]: update_settings tran:9
[Voice Changer] update configuration: tran 8
[Voice Changer][RVC]: update_settings tran:8
[Voice Changer] update configuration: tran 8
[Voice Changer][RVC]: update_settings tran:8
[Voice Changer] update configuration: tran 7
[Voice Changer][RVC]: update_settings tran:7
[Voice Changer] update configuration: tran 6
[Voice Changer][RVC]: update_settings tran:6
[Voice